In [3]:
import pandas as pd

class TransformExcel:
    def __init__(self, name_file, sheet='sb'):
        self.name_file = name_file
        self.df = pd.read_excel(f'./files/raw/{name_file}', sheet_name=sheet)
        self.run()
    

    def getDate(self):
        self.df['date'] = self.df.iloc[1,5].strftime("%Y-%m-%d")
    
    def getGP(self):
        self.df['gp'] = self.df.iloc[0,7]


    def fillLastLocal(self):
        self.df.reset_index(inplace=True)
        self.df.drop(['index'], axis=1, inplace=True)

        for i in range(1,len(self.df)):
            if pd.isna(self.df.loc[i, 'pb']):
                self.df.loc[i, 'pb'] = self.df.loc[i-1, 'pb']

            if pd.isna(self.df.loc[i, 'eq']):
                self.df.loc[i, 'eq'] = self.df.loc[i-1, 'eq']

            if pd.isna(self.df.loc[i, 'road']):
                self.df.loc[i, 'road'] = self.df.loc[i-1, 'road']

    def formatColumns(self):
        cols = list(self.df.columns)
        date_position = cols.index('date')
        gp_position = cols.index('gp')
        self.df = self.df.loc[self.df.iloc[:,9]==1,:].iloc[:,[0,1,2,3,4,5,7,9, date_position, gp_position]]
        self.df.columns = ['road','roadByworker','begin','end','pb','eq','worker','rownTrue', 'date', 'gp']
        self.df = self.df[['gp', 'date', 'road','roadByworker','begin','end','pb','eq','worker','rownTrue']]
    
    def transform_to_str(self):
        for c in self.df.columns:
            self.df[c] = self.df[c].apply(lambda x: str(x).strip().lower() if x!=None else None)
    
    def mapAreasbyRoad(self):
        self.df['areas'] = self.df[self.df['road'].str.len()>=3]['road'].apply(lambda x: x[0] if x[0] in ['1','2','3','4','5','6'] else x[:3])
        self.df.loc[self.df['pb'].isin(['selve - supervisão','sevop', 'selve - retiro','selve - dom avelar','selve - orlando gomes' , 'serat','sefit']),'areas'] = 'interno'

        self.df.loc[self.df['road'].str.len()<3,'areas'] = 'inteligencia'

    @staticmethod
    def save(df, nameCSV, cols=None):
        if cols ==None:
            df.to_csv(f'./files/db_format/{nameCSV}.csv')
        else:
            df[cols].to_csv(f'./files/db_format/{nameCSV}.csv')
    
    def run(self):
        self.getDate()
        self.getGP()
        self.formatColumns()
        self.fillLastLocal()
        self.transform_to_str()
        self.mapAreasbyRoad()
    
    def getDF(self):
        return self.df

**Execute transformations in excels**

In [4]:
import os

excelFiles = list(os.listdir('./files/raw/'))
for fds in ['sb']:
    df = TransformExcel(excelFiles[0], sheet=fds).getDF()
    for xlsx in excelFiles[1:]:
        try:
            df = pd.concat([df,TransformExcel(xlsx, sheet=fds).getDF()])
        except ValueError:
            print(xlsx)

    TransformExcel.save(df, f'escala_{fds}', ['gp', 'date', 'areas', 'road', 'begin', 'end', 'pb', 'eq', 'worker'])

/home/sandro/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/sandro/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/sandro/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/sandro/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/sandro/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/sandro/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Fo

Create Inser SQL

In [19]:
import pandas as pd

#df = pd.read_csv('./files/db_format/escala_sb.csv')

id = 2
c = list(df.columns)
for line in range(len(df)):
    l = f'INSERT INTO "escala_escala" ("id","date","road","begin","end","foothold","worker","zone","equipment","group") VALUES ({id},\'{df.iloc[line,c.index("date")]}\',\'{df.iloc[line,c.index("road")]}\',\'{df.iloc[line,c.index("begin")]}\',\'{df.iloc[line,c.index("end")]}\',\'{df.iloc[line,c.index("pb")]}\',\'{df.iloc[line,c.index("worker")]}\',\'{df.iloc[line,c.index("areas")]}\',\'{df.iloc[line,c.index("eq")]}\',\'{df.iloc[line,c.index("gp")]}\');'
    with open('./files/db_format/arq.sql', 'a') as file:
        file.write(l)
        file.write('\n')
    id = id+1

with open('./files/db_format/arq.sql', 'a') as file:
        file.write('COMMIT;')

In [16]:
line = 0
c = list(df.columns)
l = f'INSERT INTO "escala_escala" ("id","date","road","begin","end","foothold","worker","zone","equipment","group") VALUES ({id},\'{df.iloc[line,c.index("date")]}\',\'{df.iloc[line,c.index("road")]}\',\'{df.iloc[line,c.index("begin")]}\',\'{df.iloc[line,c.index("end")]}\',\'{df.iloc[line,c.index("pb")]}\',\'{df.iloc[line,c.index("worker")]}\',\'{df.iloc[line,c.index("areas")]}\',\'{df.iloc[line,c.index("eq")]}\',\'{df.iloc[line,c.index("gp")]}\');'
print(l)

INSERT INTO "escala_escala" ("id","date","road","begin","end","foothold","worker","zone","equipment","group") VALUES (2,2022-10-08,1,07:00:00,19:00:00,coordenador geral,antonio neri dos santos,inteligencia,vtr crd,i);
